<a href="https://colab.research.google.com/github/VandanC/Neuramonks_Assignment/blob/main/Task_Revision_for_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain

In [ ]:
!pip install pymupdf

In [ ]:
!pip install -U langchain-text-splitters

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install sentence_transformers

In [7]:
import fitz

raw_text = []

doc = fitz.open('API SPEC 5L-46th edition 2018.pdf')

for page in doc:
    raw_text.append(page.get_text())

raw_text = " ".join(raw_text)

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document


text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500, chunk_overlap=0
        )
texts = text_splitter.split_text(raw_text)
docs = [Document(page_content=t) for t in texts]

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

vector_embeddings = FAISS.from_documents(
            documents=docs,
            embedding=HuggingFaceBgeEmbeddings()
        )


In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
                        Question: {question}
                        Context: {context}
                        Answer:"""


prompt = PromptTemplate(
            input_variables=["context", "question"], template=prompt_template
        )




llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation"
)


qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vector_embeddings.as_retriever(
                search_type="similarity"
            ),
            return_source_documents=True,
            chain_type_kwargs={"prompt": prompt},
        )


In [ ]:
qa.invoke({'query': "Explain Guided-bend Test"})